# Code for the empirical analysis in the paper 'Growth and fund structures'
### Constantinos Kardaras; Hyeng Keun Koo; Johannes Ruf
### July 2025

To run this code, the user must have access to WRDS, along with the Python package `wrds`.  For more details on WRDS and how to interact with WRDS, see the course notes on https://github.com/johruf/CRSP_on_WRDS_introduction.

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import wrds   # needs to be installed, e.g. via pip

In [2]:
WRDS_LOGIN = 'xxx'    # update to your login info on WRDS

## Getting data

In [ ]:
def load_data():
    """Load data from WRDS"""
    try:
        with wrds.Connection(wrds_username=WRDS_LOGIN) as db:
            df_idx = db.raw_sql("SELECT caldt AS date, vwretd, totcnt FROM crsp.dsp500_v2", 
                               date_cols='date').set_index('date')
            df_rf = db.raw_sql("SELECT date, rf FROM ff.factors_daily", 
                              date_cols='date').set_index('date')
        return df_idx, df_rf
    except Exception as e:
        print(f"Error loading data: {e}")
        raise

df_idx, df_rf = load_data()

Loading library list...
Done


`vwretd` corresponds to CRSP's Value-Weighted Return Index (which reinvests dividends).

`rf` is from the Fama-French data, and uses 1M treasury rates. Data are only available beginning July 1st, 1926.

In [ ]:
df_idx.head()

In [ ]:
df_rf.head()

In [ ]:
# Getting rid of data that were not available at the time when the paper was written
CUTOFF_DATE = '2024-12-31'

In [ ]:
df = df_idx.merge(df_rf, how='left', left_index=True, right_index=True)['1926-06-30':CUTOFF_DATE]

In [ ]:
df.head(3)

In [ ]:
df.tail(3)

In [ ]:
# Calculate wealth processes
df['W_risky'] = (1 + df['vwretd']).cumprod()
df['W_risky'] /= df['W_risky'].iloc[0]

df['W_rf'] = (1 + df['rf']).cumprod()
df.at[df.index[0], 'W_rf'] = 1.0

df['W_disc'] = df['W_risky'] / df['W_rf']
df['disc_ret'] = df['W_disc'] / df['W_disc'].shift(1) - 1

df['C'] = (df['disc_ret']**2).cumsum()
df['R'] = df['disc_ret'].cumsum()

# Remove first day (no return data)
df = df['1926-07-01':]

In [ ]:
df.head(3)

In [ ]:
df.tail(3)

## Auxiliary functions

In [ ]:
def shrinkage_factor(nuhat, df, start_of_plot=0, start_of_data=-1, α=[], β=[]):    
    """
    Calculate shrinkage factor for one-fund model 
    alpha and beta parameters in case of restricted Gaussian prior 

    Based on: shrink(x) = 1 - 3 / (1 + tmp^(2/3) + tmp^(-2/3))
    where tmp = sqrt(1+x) + sqrt(x)
    """
    def shrink(x):
        tmp = np.sqrt(1 + x) + np.sqrt(x)
        return 1 - 3 / (1 + tmp**(2/3) + tmp**(-2/3))

    C = df['C'].iloc[start_of_plot:]
    if start_of_data > -1:
        C = C - df['C'].iloc[start_of_data]
    
    if len(α) == 0:
        κ = 1/C
    else:
        tmp_num = stats.norm.cdf(β) - stats.norm.cdf(α)
        κ = 1/C * (1 + (- β * stats.norm.pdf(β) + α * stats.norm.pdf(α))/tmp_num - 
                     ((stats.norm.pdf(β) - stats.norm.pdf(α)) / tmp_num)**2)
    
    ψ = 27 / 8 * nuhat**2 / κ
    return shrink(ψ)

In [ ]:
def plot_wealth(df, start=-1, ttitle=''):
    if start == -1:
        start = 0
    (df.iloc[start:] / df.iloc[start]).plot(logy=True, title=ttitle);

In [ ]:
def plot_panel(df, start_of_plot=0, start_of_data=-1, restricted=0, ttitle='', filename=''):
    if start_of_plot <= start_of_data:
        raise ValueError("start_of_plot must be greater than start_of_data")

    R = df['R'].iloc[start_of_plot:]
    C = df['C'].iloc[start_of_plot:] 
    if start_of_data > -1:
        R = R - df['R'].iloc[start_of_data]
        C = C - df['C'].iloc[start_of_data]

    
    if restricted == 0:
        nuhat = R / C
        sf = shrinkage_factor(nuhat, df, start_of_plot, start_of_data)
    else:
        a, b = restricted
        sqrt_C = np.sqrt(C)
        α = sqrt_C * (a - R / C)
        β = sqrt_C * (b - R / C)
        nuhat = R / C + (1 / sqrt_C) * (stats.norm.pdf(α) - stats.norm.pdf(β)) / (stats.norm.cdf(β) - stats.norm.cdf(α))
        sf = shrinkage_factor(nuhat, df, start_of_plot, start_of_data, α, β)


    # Calculate wealth processes
    W_nuhat = (df['disc_ret'] * nuhat.shift(1).fillna(0) + 1).cumprod()
    W_nuhat = W_nuhat.iloc[start_of_plot:] / W_nuhat.iloc[start_of_plot]
    
    W_nuhat_s = (df['disc_ret'] * (nuhat * sf).shift(1).fillna(0) + 1).cumprod()
    W_nuhat_s = W_nuhat_s.iloc[start_of_plot:] / W_nuhat_s.iloc[start_of_plot]
    
    W_disc = df['W_disc'].iloc[start_of_plot:] / df['W_disc'].iloc[start_of_plot]

    # Calculate F (maximal achievable growth)
    dC = df['C'].diff()
    theta_squared = (df['R'] / df['C']).shift(1).fillna(0) ** 2
    F = (dC * theta_squared).cumsum() / 2
    F = F.iloc[start_of_plot:] - F.iloc[start_of_plot]

    

    fig, ax = plt.subplots(2, 2, figsize=(15, 12))
    
    # Panel [0,0]: θ-hat and shrunk θ-hat
    ax[0, 0].plot(nuhat, 'b', label=r"$\hat{\theta}$", linewidth=1.5)
    ax[0, 0].plot(nuhat * sf, 'r', label=r"$a \hat{\theta}$", linewidth=1.5)
    ax[0, 0].set_title(r'Estimate of $\hat{\theta}$ in the Bayesian setting')
    ax[0, 0].legend(frameon=False)
    
    # Panel [1,0]: shrinkage factor
    ax[1, 0].plot(sf, 'r', label=r"$a$", linewidth=1.5)
    ax[1, 0].set_title(r'Shrinkage factor $a$')
    ax[1, 0].legend(frameon=False)
    
    # Panel [0,1]: wealth processes
    ax[0, 1].plot(np.log(W_nuhat), 'b', linewidth=1.5, label=r'Log-wealth, using $\hat{\theta}$')
    ax[0, 1].plot(np.log(W_nuhat_s), 'r', linewidth=1.5, label=r'Log-wealth, using $a \hat{\theta}$')
    ax[0, 1].plot(np.log(W_disc), 'k', linewidth=1.5, label=r'Log-market portfolio')
    ax[0, 1].plot(F, 'g', linewidth=1.5, label=r'Maximal $\mathcal{F}$-achievable growth $F$')
    ax[0, 1].set_title('Logarithm of wealth processes')
    ax[0, 1].legend(frameon=False, fontsize=9)
    
    # Panel [1,1]: Integrated variance
    ax[1, 1].plot(C, 'm', linewidth=1.5, label=r'$C_{ww}$')
    ax[1, 1].set_title(r'Integrated variance $C_{ww}$')
    ax[1, 1].legend(frameon=False)

    
    plt.suptitle(ttitle, fontsize=14)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    
    if filename:
        # Create plots directory if it doesn't exist
        import os
        os.makedirs('plots', exist_ok=True)
        fig.savefig(f'plots/{filename}.png')
        print(f"Plot saved as plots/{filename}.png")

## Preliminary analysis and dummy checks

In [ ]:
print(f"Date range: {df.index[0]} to {df.index[-1]}")
print(f"Total observations: {len(df)}")

In [ ]:
plot_wealth(df[['W_risky', 'W_disc', 'W_rf']], ttitle='Various wealth processes')

Let's compute the discounted return differently, via excess returns:

In [ ]:
rtrn_excess = df['vwretd'] - df['rf']
df['W_excess'] = (rtrn_excess.fillna(0) + 1).cumprod()
df['W_excess'] = df['W_excess'] 

In [ ]:
plot_wealth(df[['W_disc', 'W_excess']], ttitle='Dummy check: compute discounted wealth index in different ways')

In [ ]:
df[['W_disc', 'W_excess']].tail(3)

In [ ]:
if 'W_excess' in df.columns:
    del df['W_excess']

Let's compute integrated variance (via quadratic variation) differently:

In [ ]:
df['QV_raw'] = (df['vwretd']**2).cumsum()

In [ ]:
df[['QV_raw', 'C']].plot(title='Quadratic variation (raw and discounted returns)');

In [ ]:
df[['QV_raw', 'C']].tail(3)

In [ ]:
if 'QV_raw' in df.columns:
    del df['QV_raw']

## Creating the panel for the paper

In [ ]:
df.iloc[7368:7372]

In [ ]:
idx_July1951 = 7370

In [ ]:
df.index[idx_July1951]

In [ ]:
plot_panel(df, start_of_plot=idx_July1951, filename='US')

## Some robustness checks

Let's use data only after 1957, March 1, when the index contains around 500 stocks.

In [ ]:
df['totcnt'].plot(title='Number of stocks in index');

In [ ]:
idx_500 = 8825 
df.iloc[idx_500-1:].head(4)

If only data are used after 500 stocks are included, again starting the plot 7370 trading days later.

In [ ]:
plot_panel(df, start_of_plot=idx_500+7370, start_of_data=idx_500, ttitle='US (using data after 500 stocks introduced)')

Now restrict $\nu$ to lie in a certain range $(a,b)$, via a restricted Gaussian prior.

In [ ]:
bounds = [0.5, 3]
plot_panel(df, start_of_plot=idx_July1951, restricted=bounds, ttitle='US (nuhat restricted to [{},{}])'.format(bounds[0], bounds[1]))

### Dummy check: Maximal achievable $\mathcal F$ growth

In [ ]:
start = idx_500

In [ ]:
dC = df['C'].diff()
F = (dC * ((df['R'] / df['C']).shift(1))**2).cumsum() / 2
F = F.iloc[start:] - F.iloc[start]

Alternative computation via half of quadratic variation of log wealth:

In [ ]:
nuhat = df['R'] / df['C']
W_nuhat = (df['disc_ret'] * nuhat.shift(1) + 1).fillna(1).cumprod()

tmp = np.log(W_nuhat.iloc[start:] /  W_nuhat.iloc[start])
QV_log_wealth = (tmp.diff()**2).cumsum()/2
QV_log_wealth.iloc[0] = 0.0

In [ ]:
F.plot()
QV_log_wealth.plot()
plt.legend(['Maximal F-achievable growth', 'Half of quadratic variation of log wealth']);

Difference appears on Black Monday (19 Oct 1987), when the market exhibits a huge negative return that makes the normal approximation invalid.

In [ ]:
df[idx_500+7700:].head()